# Imports

In [1]:
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.insert(0, "lib/")
from data.coco_dataset import CocoDataset
from utils.preprocess_sample import preprocess_sample
from utils.collate_custom import collate_custom
from utils.utils import to_cuda_variable
import utils.result_utils as result_utils
from utils.json_dataset_evaluator import evaluate_boxes,evaluate_masks
from model.detector import detector

torch_ver = torch.__version__[:3]

# Parameters

In [2]:
# Pretrained model
# https://s3-us-west-2.amazonaws.com/detectron/35858828/12_2017_baselines/e2e_mask_rcnn_R-50-C4_2x.yaml.01_46_47.HBThTerB/output/train/coco_2014_train%3Acoco_2014_valminusminival/generalized_rcnn/model_final.pkl
arch='resnet50'
pretrained_model_file = 'files/trained_models/mask/model_final.pkl'
mapping_file = 'files/mapping_files/resnet50_mapping.npy'

# COCO minival2014 dataset path
coco_ann_file='datasets/data/coco/annotations/instances_minival2014.json'
img_dir='datasets/data/coco/val2014'

# Create dataset

In [3]:
dataset = CocoDataset(ann_file=coco_ann_file,img_dir=img_dir,
                       sample_transform=preprocess_sample(target_sizes=[800]))
dataloader = DataLoader(dataset, batch_size=1, # only batch_size=1 is supported by now
                        shuffle=False, num_workers=0, collate_fn=collate_custom)

loading annotations into memory...
Done (t=1.12s)
creating index...
index created!


# Create detector model

In [4]:
model = detector(arch=arch,
                 detector_pkl_file=pretrained_model_file,
                 mapping_file=mapping_file,
                 use_rpn_head = True,
                 use_mask_head = True)
model = model.cuda()

loading pretrained weights


# Evaluate

In [5]:
# Create data structure to store results
all_boxes, all_segms, all_keyps = result_utils.empty_results(dataset.num_classes, len(dataset)) 
# (only all_boxes will be used for fast RCNN)

In [6]:
# Compute detections for whole dataset
for i, batch in enumerate(dataloader):
    batch = to_cuda_variable(batch)
    # forward pass
    if torch_ver=="0.4": # handle change in "volatile"
        with torch.no_grad():
            class_scores,bbox_deltas,rois,img_features=model(batch['image'],
                                                             scaling_factor=batch['scaling_factors'])   
    else:
        class_scores,bbox_deltas,rois,img_features=model(batch['image'],
                                                             scaling_factor=batch['scaling_factors'])   
    # postprocess output:
    # - convert coordinates back to original image size, 
    # - treshold proposals based on score,
    # - do NMS.
    scores_final, boxes_final, boxes_per_class = result_utils.postprocess_output(rois,
                                                                    batch['scaling_factors'],
                                                                    batch['original_im_size'],
                                                                    class_scores,
                                                                    bbox_deltas)
    if len(boxes_final)==0:
        continue
        
    # compute masks
    boxes_final_th = Variable(torch.cuda.FloatTensor(boxes_final))*batch['scaling_factors']
    masks=model.mask_head(img_features,boxes_final_th)
    # postprocess mask output:
    h_orig = int(batch['original_im_size'].squeeze()[0].data.cpu().numpy().item())
    w_orig = int(batch['original_im_size'].squeeze()[1].data.cpu().numpy().item())
    cls_segms = result_utils.segm_results(boxes_per_class, masks.cpu().data.numpy(), boxes_final, h_orig, w_orig)
    
    # store results
    result_utils.extend_results(i, all_boxes, boxes_per_class)
    result_utils.extend_results(i, all_segms, cls_segms)
    
    if i%100==0:
        print("{}/{}".format(i+1,len(dataset)))

print('Done!')

1/5000
101/5000
201/5000
301/5000
401/5000
501/5000
601/5000
701/5000
801/5000
901/5000
1001/5000
1101/5000
1201/5000
1301/5000
1401/5000
1501/5000
1601/5000
1701/5000
1801/5000
1901/5000
2001/5000
2101/5000
2201/5000
2301/5000
2401/5000
2501/5000
2601/5000
2701/5000
2801/5000
2901/5000
3001/5000
3101/5000
3201/5000
3301/5000
3401/5000
3501/5000
3601/5000
3701/5000
3801/5000
3901/5000
4001/5000
4101/5000
4201/5000
4301/5000
4401/5000
4501/5000
4601/5000
4701/5000
4801/5000
4901/5000
Done!


In [10]:
# # Save detection and segmentation results
np.save('files/results/all_boxes_segms_mask.npy',{'all_boxes': all_boxes, 'all_segms': all_segms})

In [8]:
# Compute evaluation metrics
coco_box_eval = evaluate_boxes(json_dataset=dataset.coco, 
                           all_boxes=all_boxes, 
                           output_dir='files/results/',
                           use_salt=False, cleanup=False)

Loading and preparing results...
DONE (t=1.53s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=39.89s).
Accumulating evaluation results...
DONE (t=5.97s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.378
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.581
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.410
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.207
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.419
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.521
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.317
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.485
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.503
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [9]:
coco_segm_eval = evaluate_masks(json_dataset=dataset.coco, 
                           all_boxes=all_boxes,
                           all_segms=all_segms,
                           output_dir='files/results/',
                           use_salt=False, cleanup=False)

Loading and preparing results...
DONE (t=4.94s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=44.14s).
Accumulating evaluation results...
DONE (t=6.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.328
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.545
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.345
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.133
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.360
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.511
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.284
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.427
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.442
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1